# 2 Build Up Dataset of Player (including skin tone)

<h3>Scrape data of player from SoccerWiki</h3>

<h4>2022.01.18 The final (latest) version is FinalPlayerData.csv</h4>

In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
# we can tap something in the search bar
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import time
import parsel
import pandas as pd
import requests

In [2]:
# extract skin tone of player from player page
# automatically search the name of player in the SoccerWiki and return the skin color of player
def extractSkinTone(name):
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get("https://en.soccerwiki.org/wiki.php")
    print(driver.title)
    
    search = driver.find_element_by_name("q")
    search.send_keys(name)

    driver.implicitly_wait(5)
    searchResult = driver.find_element_by_xpath("//*[@class='tabledata']/tbody/tr/td/a").click()

    html_data = parsel.Selector(driver.page_source)
    skinTone = html_data.xpath("//*[@id='realLifeTable']/tbody/tr[10]/td/text()").extract()
    
    print(skin)


In [2]:
# Extract club links from league/competitions page
from re import search
def extractClubLinks(leagueLink):
    r = requests.get(leagueLink)
    html_data = parsel.Selector(r.text)
#     clubs = html_data.xpath("//*[@summary='All clubs']/tr")
    clubs = html_data.xpath("//div[@class='col-lg-8 col-12']//table[@class='table-custom table-roster']/tbody/tr")
#     clubs = html_data.xpath("/html/body/div[2]/section/div/div[2]/div[1]/div[1]/table/tbody/tr")
    hrefs = []
    for c in clubs:
        item = c.xpath("./td[2]/a/@href").extract()
        if len(item) != 0:
            hrefs.append("https://en.soccerwiki.org/" + item[0])
    
    # remove referee links
    for h in hrefs:
        if "refid" in h:
            hrefs.remove(h)
        hrefs = hrefs[:]
    return hrefs 

In [68]:
# extract links of club in the Premier League
# 28 Premier League
# 89 La Liga
# 51 Serie A
# 39 Bundesliga
# 36 Ligue 1
# 309 Division di Honor
# 78 Premier Liga
# 14 Searie A
# 62 Liga MX
# 95 Süper Lig
# 10 First Division A
# 98 Major League Soccer
# 29 Football League Championship
# 93 Super League
# 66 Eredivisie
# 40 2. Bundesliga
# clubs = extractClubLinks("https://en.soccerwiki.org/league.php?leagueid=28") 
# clubs = extractClubLinks("https://en.soccerwiki.org/league.php?leagueid=89")
# clubs = extractClubLinks("https://en.soccerwiki.org/league.php?leagueid=51")
# clubs = extractClubLinks("https://en.soccerwiki.org/league.php?leagueid=39") 
# clubs = extractClubLinks("https://en.soccerwiki.org/league.php?leagueid=36") 
# clubs = extractClubLinks("https://en.soccerwiki.org/league.php?leagueid=309")
# clubs = extractClubLinks("https://en.soccerwiki.org/league.php?leagueid=78")
# clubs = extractClubLinks("https://en.soccerwiki.org/league.php?leagueid=14")


# clubs = extractClubLinks("https://en.soccerwiki.org/league.php?leagueid=62")
# clubs = extractClubLinks("https://en.soccerwiki.org/league.php?leagueid=95")
# clubs = extractClubLinks("https://en.soccerwiki.org/league.php?leagueid=10")
# clubs = extractClubLinks("https://en.soccerwiki.org/league.php?leagueid=98")
# clubs = extractClubLinks("https://en.soccerwiki.org/league.php?leagueid=29")
# clubs = extractClubLinks("https://en.soccerwiki.org/league.php?leagueid=93")
# clubs = extractClubLinks("https://en.soccerwiki.org/league.php?leagueid=66")
clubs = extractClubLinks("https://en.soccerwiki.org/league.php?leagueid=40")
clubs

['https://en.soccerwiki.org//squad.php?clubid=1513',
 'https://en.soccerwiki.org//squad.php?clubid=386',
 'https://en.soccerwiki.org//squad.php?clubid=394',
 'https://en.soccerwiki.org//squad.php?clubid=397',
 'https://en.soccerwiki.org//squad.php?clubid=1287',
 'https://en.soccerwiki.org//squad.php?clubid=398',
 'https://en.soccerwiki.org//squad.php?clubid=1109',
 'https://en.soccerwiki.org//squad.php?clubid=481',
 'https://en.soccerwiki.org//squad.php?clubid=401',
 'https://en.soccerwiki.org//squad.php?clubid=402',
 'https://en.soccerwiki.org//squad.php?clubid=403',
 'https://en.soccerwiki.org//squad.php?clubid=1512',
 'https://en.soccerwiki.org//squad.php?clubid=1437',
 'https://en.soccerwiki.org//squad.php?clubid=406',
 'https://en.soccerwiki.org//squad.php?clubid=483',
 'https://en.soccerwiki.org//squad.php?clubid=1751',
 'https://en.soccerwiki.org//squad.php?clubid=1435',
 'https://en.soccerwiki.org//squad.php?clubid=414']

In [14]:
# Extract player links from the club page
def extractPlayerLinks(clubLink):
    r = requests.get(clubLink)
    html_data = parsel.Selector(r.text)
    players = html_data.xpath("//*[@id='datatable']/tbody/tr")
    hrefs = []
    for p in players:
        item = p.xpath("//td[@class='text-left']/a/@href").extract()
    if len(item) != 0:
        for i in range(len(item)):
            hrefs.append("https://en.soccerwiki.org/" + item[i])
    # remove referee links
    for h in hrefs:
        if "pid" not in h:
            hrefs.remove(h)
        hrefs = hrefs[:]    
    return hrefs

In [69]:
# obtain all links of player from the clubs of league
players = []
for c in clubs:
    players.extend(extractPlayerLinks(c))  
len(players)
players

['https://en.soccerwiki.org//player.php?pid=47130',
 'https://en.soccerwiki.org//player.php?pid=51754',
 'https://en.soccerwiki.org//player.php?pid=113348',
 'https://en.soccerwiki.org//player.php?pid=78765',
 'https://en.soccerwiki.org//player.php?pid=42203',
 'https://en.soccerwiki.org//player.php?pid=80978',
 'https://en.soccerwiki.org//player.php?pid=58723',
 'https://en.soccerwiki.org//player.php?pid=105043',
 'https://en.soccerwiki.org//player.php?pid=85004',
 'https://en.soccerwiki.org//player.php?pid=88991',
 'https://en.soccerwiki.org//player.php?pid=105044',
 'https://en.soccerwiki.org//player.php?pid=113347',
 'https://en.soccerwiki.org//player.php?pid=98494',
 'https://en.soccerwiki.org//player.php?pid=105042',
 'https://en.soccerwiki.org//player.php?pid=98539',
 'https://en.soccerwiki.org//player.php?pid=74508',
 'https://en.soccerwiki.org//player.php?pid=74859',
 'https://en.soccerwiki.org//player.php?pid=105045',
 'https://en.soccerwiki.org//player.php?pid=70747',
 'http

In [ ]:
# the latest and right function
# old web structure of soccerwiki
# def ObtainPlayerData(playerLink):
#     r = requests.get(playerLink)
#     r.raise_for_status()
#     html_data = parsel.Selector(r.text)
#     templist = []
#     try:
#         name = html_data.xpath("//*[@id='realLifeTable']/tr[1]/td/text()").extract()[0]
#         print(name)
#         club = html_data.xpath("//*[@id='clubnamediv']/a/text()").extract()[0]
#         birthDate = html_data.xpath("//*[@id='realLifeTable']/tr[4]/td/text()").extract()[0]
#         nation = html_data.xpath("//*[@id='realLifeTable']/tr[5]/td/a/text()").extract()[0]
#         height = html_data.xpath("//*[@id='realLifeTable']/tr[6]/td/text()").extract()[0]
#         weight = html_data.xpath("//*[@id='realLifeTable']/tr[7]/td/text()").extract()[0]
#         hairColor = html_data.xpath("//*[@id='realLifeTable']/tr[8]/td/text()").extract()[0]
#         hairStyle = html_data.xpath("//*[@id='realLifeTable']/tr[9]/td/text()").extract()[0]
#         skinColor = html_data.xpath("//*[@id='realLifeTable']/tr[10]/td/text()").extract()[0]
#         facialHair = html_data.xpath("//*[@id='realLifeTable']/tr[11]/td/text()").extract()[0]
#         squardNo = html_data.xpath("//*[@id='realLifeTable']/tr[12]/td/text()").extract()[0]
#         player_dict  = {"Name":name, "Club":club, "Age":age, "BirthDate":birthDate,
#                         "Nation": nation, "Height":height, "Weight":weight, 
#                         "HairColor":hairColor, "HairStyle":hairStyle, "SkinColor":skinColor, 
#                         "FacialHair":facialHair, "SquadNo":squadNo}
#         templist.append(player_dict)
#         df = pd.DataFrame(templist)
#         df.to_csv('PlayerDataNewLigue1.csv', mode='a', index=False, header=False)
#     except IndexError:
#         print(playerLink)
#         with open('Exception_links_Ligue1.txt','a') as f:
#             f.write(playerLink)

In [70]:
# new web structure of soccerwiki
def ObtainPlayerData(playerLink):
    
    r = requests.get(playerLink)
    r.raise_for_status()
    html_data = parsel.Selector(r.text)
    templist = []
    try:
        name = html_data.xpath("//*[@class='col-md-9 col-7']/p[1]/text()").extract()[0].strip()
        print(name)
        nation = html_data.xpath("//*[@class='col-md-9 col-12']/p[1]/a[2]/text()").extract()[0]
        height = html_data.xpath("//*[@class='col-md-9 col-12']/p[2]/text()").extract()[0].strip()
        weight = html_data.xpath("//*[@class='col-md-9 col-12']/p[3]/text()").extract()[0].strip()
        club = html_data.xpath("//*[@class='col-md-9 col-12']/p[4]/a/text()").extract()[0].strip()
        print(club)
        hairColor = html_data.xpath("//*[@class='col-md-9 col-12']/p[7]/text()").extract()[0].strip()
        hairStyle = html_data.xpath("//*[@class='col-md-9 col-12']/p[8]/text()").extract()[0].strip()
        skinColor = html_data.xpath("//*[@class='col-md-9 col-12']/p[9]/text()").extract()[0].strip()
        facialHair = html_data.xpath("//*[@class='col-md-9 col-12']/p[10]/text()").extract()[0].strip()
        squadNo = html_data.xpath("//*[@class='col-md-9 col-12']/p[5]/text()").extract()[0].strip()
        player_dict  = {"Name":name, "Club":club,
                        "Nation": nation, "Height":height, "Weight":weight, 
                        "HairColor":hairColor, "HairStyle":hairStyle, "SkinColor":skinColor, 
                        "FacialHair":facialHair, "SquadNo":squadNo}
        templist.append(player_dict)
        df = pd.DataFrame(templist)
        df.to_csv('PlayerData2Bundesliga.csv', mode='a', index=False, header=False)
    except IndexError:
        print(playerLink)
        with open('Exception_links_2Bundesliga.txt','a') as f:
            f.write(playerLink)

In [71]:
# obtain the player data from player page
for p in players:
    ObtainPlayerData(p)

Kevin Müller
1. FC Heidenheim 1846
Vitus Eicher
1. FC Heidenheim 1846
Paul Tschernuth
1. FC Heidenheim 1846
Oliver Hüsing
1. FC Heidenheim 1846
Norman Theuerkauf
1. FC Heidenheim 1846
Jonas Föhrenbach
1. FC Heidenheim 1846
Andreas Geipl
1. FC Heidenheim 1846
Melvin Ramusović
1. FC Heidenheim 1846
Patrick Mainka
1. FC Heidenheim 1846
Dženis Burnić
1. FC Heidenheim 1846
Jan Schöppner
1. FC Heidenheim 1846
Tim Seifert
1. FC Heidenheim 1846
Kevin Simone Sessa
1. FC Heidenheim 1846
Marvin Lee Rittmüller
1. FC Heidenheim 1846
Tobias Mohr
1. FC Heidenheim 1846
Konstantin Kerschbaumer
1. FC Heidenheim 1846
Marnon-Thomas Busch
1. FC Heidenheim 1846
Julian Stark
1. FC Heidenheim 1846
Tim Kleindienst
1. FC Heidenheim 1846
Stefan Schimmer
1. FC Heidenheim 1846
Denis Thomalla
1. FC Heidenheim 1846
Robert Leipertz
1. FC Heidenheim 1846
Florian Pick
1. FC Heidenheim 1846
Christian Kühlwetter
1. FC Heidenheim 1846
Gianni Mollo
1. FC Heidenheim 1846
Christian Mathenia
1. FC Nürnberg
Carl Klaus
1. FC Nü

Franck Junior Evina
Hannover 96
Philipp Ochs
Hannover 96
Valmir Sulejmani
Hannover 96
Markus Kolke
Hansa Rostock
Luis Klatte
Hansa Rostock
Ben-Alexander Voll
Hansa Rostock
Thomas Meißner
Hansa Rostock
Ryan Patrick Malone
Hansa Rostock
Damian Roßbach
Hansa Rostock
Julian Riedel
Hansa Rostock
Lukas Scherff
Hansa Rostock
Svante Ulf Ingel Ingelsson
Hansa Rostock
Hanno Behrens
Hansa Rostock
Björn Rother
Hansa Rostock
Simon Rhein
Hansa Rostock
Tobias Schwede
Hansa Rostock
Calogero Rizzuto
Hansa Rostock
Nico Neidhart
Hansa Rostock
Bentley Baxter Bahn
Hansa Rostock
Nik Omladič
Hansa Rostock
Maurice Jerome Litka
Hansa Rostock
John Verhoek
Hansa Rostock
Ridge Munsy
Hansa Rostock
Pascal Breier
Hansa Rostock
Kevin Schumacher
Hansa Rostock
Theo Gunnar Martens
Hansa Rostock
Haris Duljević
Hansa Rostock
Ioannis Gelios
Holstein Kiel
Thomas Dähne
Holstein Kiel
Timon Moritz Weiner
Holstein Kiel
Stefan Thesker
Holstein Kiel
Marco Komenda
Holstein Kiel
Mikkel Kirkeskov Andersen
Holstein Kiel
Hauke Finn Wa

In [72]:
# add header to the csv file 
df = pd.read_csv("PlayerData2Bundesliga.csv", 
                  names=["Name", "Club", "Nation",
                        "Height","Weight","HairColor",
                        "HairStyle","SkinColor","FacialHair","SquadNo"])
df

,Name,Club,Nation,Height,Weight,HairColor,HairStyle,SkinColor,FacialHair,SquadNo
0,Kevin Müller,1. FC Heidenheim 1846,Germany,188cm,87kg,Black,Cropped,White,Clean,1
1,Vitus Eicher,1. FC Heidenheim 1846,Germany,190cm,80kg,Brown,Short,White,Clean,22
2,Paul Tschernuth,1. FC Heidenheim 1846,Austria,186cm,77kg,Blonde,Short,White,Clean,34
3,Oliver Hüsing,1. FC Heidenheim 1846,Germany,193cm,88kg,Dark Brown,Short,White,Clean,5
4,Norman Theuerkauf,1. FC Heidenheim 1846,Germany,183cm,75kg,Dark Brown,Short,White,Stubble,30
...,...,...,...,...,...,...,...,...,...,...
464,Jean-Manuel Mbom,Werder Bremen,Germany,183cm,75kg,Black,Cropped,Brown,Stubble,34
465,Niclas Füllkrug,Werder Bremen,Germany,189cm,83kg,Blonde,Short,White,Stubble,11
466,Marvin Ducksch,Werder Bremen,Germany,188cm,75kg,Brown,Cropped,White,Stubble,7
467,Nick Woltemade,Werder Bremen,Germany,196cm,76kg,Brown,Short,White,Clean,29


In [ ]:
# add data(in the exception links) manually
# df = df.append({"Name":"Francisco António Machado Mota Castro Trincão", "Club":"Barcelona", "Age":21, 
#                 "BirthDate":"29 December 1999", "Nation": "Portugal", 
#                 "Height":184, "Weight":75, "HairColor":"Black", "HairStyle":"Short", 
#                 "SkinColor":"Olive", "FacialHair":"Goatee", "SquadNo":11}
#                 , ignore_index=True)

In [73]:
# save player data of each league into respective csv file
df.to_csv("2BundesligaPlayerData.csv",index=False)

In [75]:
# combine player data from different leagues
dff1 = pd.read_csv("Ligue1PlayerData.csv")
dff2 = pd.read_csv("PremierPlayerData.csv")
dff3 = pd.read_csv("LaLigaPlayerData.csv")
dff4 = pd.read_csv("BundesligaPlayerData.csv")
dff5 = pd.read_csv("DivisionDiHonorPlayerData.csv")
dff6 = pd.read_csv("PremierLigaPlayerData.csv")
dff7 = pd.read_csv("SearieAPlayerData.csv")
dff8 = pd.read_csv("LigaMXPlayerData.csv")
dff9 = pd.read_csv("SüperLigPlayerData.csv")
dff10 = pd.read_csv("FirstDivisionAPlayerData.csv")
dff11 = pd.read_csv("MajorLeagueSoccerPlayerData.csv")
dff12 = pd.read_csv("FootballLeagueChampionshipPlayerData.csv")
dff13 = pd.read_csv("SuperLeaguePlayerData.csv")
dff14 = pd.read_csv("EredivisiePlayerData.csv")
dff15 = pd.read_csv("2BundesligaPlayerData.csv")

frames = [dff1, dff2, dff3, dff4, dff5, dff6, dff7, dff8, dff9, dff10, dff11, dff12, dff13, dff14, dff15]
new_players = pd.concat(frames)
# remove repeated player data
new_players = new_players.drop_duplicates()
new_players = new_players.reset_index(drop=True)
new_players.insert(0, 'ID', range(1, len(new_players)+1))
new_players.to_csv("FinalPlayerData.csv", index=False)
new_players

,ID,Name,Club,Nation,Height,Weight,HairColor,HairStyle,SkinColor,FacialHair,SquadNo
0,1,Danijel Petković,Angers SCO,Montenegro,196cm,86kg,Black,Short,Olive,Beard,30
1,2,Anthony Mandrea,Angers SCO,France,186cm,75kg,Black,Cropped,Olive,Beard,16
2,3,Melvin Zinga,Angers SCO,France,187cm,80kg,Black,Cropped,Olive,Clean,Unknown
3,4,Romain Thomas,Angers SCO,France,193cm,84kg,Dark Brown,Short,White,Stubble,24
4,5,Ismaël Traoré,Angers SCO,Ivory Coast,185cm,75kg,Black,Cropped,Black,Beard,8
...,...,...,...,...,...,...,...,...,...,...,...
7552,7553,Jean-Manuel Mbom,Werder Bremen,Germany,183cm,75kg,Black,Cropped,Brown,Stubble,34
7553,7554,Niclas Füllkrug,Werder Bremen,Germany,189cm,83kg,Blonde,Short,White,Stubble,11
7554,7555,Marvin Ducksch,Werder Bremen,Germany,188cm,75kg,Brown,Cropped,White,Stubble,7
7555,7556,Nick Woltemade,Werder Bremen,Germany,196cm,76kg,Brown,Short,White,Clean,29
